In [14]:
import os
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from models.custom_cnn import get_custom_cnn_model
from tqdm import tqdm

In [15]:

# --- Config ---
DATA_DIR = "data/stanford_cars"
TRAIN_CSV = os.path.join(DATA_DIR, "anno_train.csv")
VAL_CSV = os.path.join(DATA_DIR, "anno_validation.csv")
NAMES_CSV = os.path.join(DATA_DIR, "names.csv")
TRAIN_IMG_DIR = os.path.join(DATA_DIR, "train")
VAL_IMG_DIR = os.path.join(DATA_DIR, "validation")
BATCH_SIZE = 64
EPOCHS = 30
LR = 0.001
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
IMG_SIZE = 128

In [16]:
# --- Dataset ---
class CarsDataset(Dataset):
    def __init__(self, anno_csv, img_dir, names_csv, transform=None, half_data=False):
        self.anno = pd.read_csv(anno_csv, header=None)
        if half_data:
            self.anno = self.anno.iloc[:len(self.anno)//4]
        self.img_dir = img_dir
        self.transform = transform
        with open(names_csv) as f:
            self.class_names = [line.strip() for line in f]
        self.num_classes = len(self.class_names)

    def __len__(self):
        return len(self.anno)

    def __getitem__(self, idx):
        row = self.anno.iloc[idx]
        img_name = row[0]
        bbox = tuple(row[1:5])
        label = int(row[5]) - 1  # CSV is 1-based
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert("RGB")
        # Optional: crop to bbox
        # image = image.crop(bbox)
        if self.transform:
            image = self.transform(image)
        return image, label

# --- Transforms ---
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# --- DataLoaders ---
train_ds = CarsDataset(TRAIN_CSV, TRAIN_IMG_DIR, NAMES_CSV, transform=train_transform, half_data=True)
val_ds = CarsDataset(VAL_CSV, VAL_IMG_DIR, NAMES_CSV, transform=val_transform)
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

In [17]:

# --- Model ---
model = get_custom_cnn_model(num_classes=train_ds.num_classes).to(DEVICE)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

# --- Training/Validation Loop ---
def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    for imgs, labels in tqdm(loader, desc="Train", leave=False):
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * imgs.size(0)
        _, preds = outputs.max(1)
        correct += preds.eq(labels).sum().item()
        total += imgs.size(0)
    return running_loss / total, correct / total

def validate(model, loader, criterion):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for imgs, labels in tqdm(loader, desc="Val", leave=False):
            imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * imgs.size(0)
            _, preds = outputs.max(1)
            correct += preds.eq(labels).sum().item()
            total += imgs.size(0)
    return running_loss / total, correct / total

# --- Main Loop ---
for epoch in range(EPOCHS):
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion)
    val_loss, val_acc = validate(model, val_loader, criterion)
    print(f"Epoch {epoch+1}/{EPOCHS} | "
          f"Train Loss: {train_loss:.4f} Acc: {train_acc:.4f} | "
          f"Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}")

# --- Save model ---
torch.save(model.state_dict(), "custom_cnn_best.pth")
print("Training complete. Model saved as custom_cnn_best.pth")

Epoch 1/30 | Train Loss: 6.4439 Acc: 0.0406 | Val Loss: 3.7723 Acc: 0.0748


Epoch 2/30 | Train Loss: 3.7665 Acc: 0.0745 | Val Loss: 3.6477 Acc: 0.0723


Epoch 3/30 | Train Loss: 3.6759 Acc: 0.0794 | Val Loss: 3.5340 Acc: 0.1094


Epoch 4/30 | Train Loss: 3.6662 Acc: 0.0929 | Val Loss: 3.5844 Acc: 0.1106


Epoch 5/30 | Train Loss: 3.6394 Acc: 0.0971 | Val Loss: 3.4786 Acc: 0.1119


...

Epoch 12/30 | Train Loss: 3.5744 Acc: 0.0876 | Val Loss: 3.5471 Acc: 0.1075


Epoch 13/30 | Train Loss: 3.5773 Acc: 0.0918 | Val Loss: 3.4889 Acc: 0.1119


Epoch 14/30 | Train Loss: 3.5650 Acc: 0.0865 | Val Loss: 3.4690 Acc: 0.1112


Epoch 15/30 | Train Loss: 3.5686 Acc: 0.0886 | Val Loss: 3.5095 Acc: 0.1119


KeyboardInterrupt: 

In [18]:
import torch
import torch.nn as nn
from torchvision import models

# --- Model ---
def get_resnet18_model(num_classes):
    model = models.resnet18(weights='IMAGENET1K_V1')
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model

model = get_resnet18_model(num_classes=train_ds.num_classes).to(DEVICE)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

# --- Training/Validation Loop ---
def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    for imgs, labels in tqdm(loader, desc="Train", leave=False):
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * imgs.size(0)
        _, preds = outputs.max(1)
        correct += preds.eq(labels).sum().item()
        total += imgs.size(0)
    return running_loss / total, correct / total

def validate(model, loader, criterion):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for imgs, labels in tqdm(loader, desc="Val", leave=False):
            imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * imgs.size(0)
            _, preds = outputs.max(1)
            correct += preds.eq(labels).sum().item()
            total += imgs.size(0)
    return running_loss / total, correct / total

# --- Main Loop ---
for epoch in range(EPOCHS):
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion)
    val_loss, val_acc = validate(model, val_loader, criterion)
    print(f"Epoch {epoch+1}/{EPOCHS} | "
          f"Train Loss: {train_loss:.4f} Acc: {train_acc:.4f} | "
          f"Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}")

# --- Save model ---
torch.save(model.state_dict(), "resnet18_best.pth")
print("Training complete. Model saved as resnet18_best.pth")

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/manuramirezsilva/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:20<00:00, 2.27MB/s]


Epoch 1/30 | Train Loss: 3.5545 Acc: 0.1091 | Val Loss: 3.9094 Acc: 0.1044


Epoch 2/30 | Train Loss: 3.2153 Acc: 0.1557 | Val Loss: 3.5959 Acc: 0.1168


Epoch 3/30 | Train Loss: 3.0386 Acc: 0.1917 | Val Loss: 3.3484 Acc: 0.1675


...

Epoch 8/30 | Train Loss: 2.4922 Acc: 0.3051 | Val Loss: 2.8192 Acc: 0.2330


Epoch 9/30 | Train Loss: 2.3573 Acc: 0.3446 | Val Loss: 2.7882 Acc: 0.2627


Epoch 10/30 | Train Loss: 2.2624 Acc: 0.3595 | Val Loss: 3.3989 Acc: 0.2021


Epoch 11/30 | Train Loss: 2.1692 Acc: 0.3849 | Val Loss: 2.9750 Acc: 0.2324


KeyboardInterrupt: 